In [18]:
sSparkContext()

NameError: name 'sparkContext' is not defined

In [16]:
import numpy as np
import pandas as pd
import datetime
import time
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pdb

In [17]:
# Load pickle files with saved decompositions 
# set avergagin method. options are 'mean' 'median' 'exp'


forecast = True
backtest_year  = 2014

path = {}
decomp = {}

year_list = [2013,2014]
the34_23 = '2_3'

num_of_stores = 800

for y in year_list:
    path[y] = 'data/800_2_3_xmastrial'
#path[y] = 'data/800_2_3_xmastrial'

if forecast:
     decomp[y] =  dict(sc.pickleFile(path[y]).collect())
else:
    if y < backtest_year:
        decomp[y] =  dict(sc.pickleFile(path[y]).collect())



NameError: name 'sc' is not defined

In [ ]:
#print np.where(decomp[2013]['F11W']['stores'] == 3137 )
#print np.where(decomp[2013]['F11W']['products'] == 'F11WE')

#decomp[2013]['F11W']['E_T'][411,:,3]
#decomp[2013]['F11W']['E_T']


In [ ]:
#Load the PTPs

if forecast:
        PTPs_path  = 'zg43_xmasPTPs'
else:
        PTPs_path  = 'zg43_xmasPTPs_'+str(backtest_year)

    
xmas_ptps = (sqlContext.read.format('jdbc')
                .options(url="jdbc:teradata://TDPS.ukroi.tesco.org/DATABASE=DXWI_PROD_CRORDER_PLAY_PEN",
                     dbtable=PTPs_path,
                     user = "GCT8",
                     password = "ludo4ka",
                     driver = "com.teradata.jdbc.TeraDriver")
                .load()
             )
    
    
#xmas_ptps = (sqlContext.read.format("com.databricks.spark.csv")
#                .option("header","false")
#                .option("inferSchema", "true")
#                .load(PTPs_path)
#                .toDF('retail_outlet_number','product','calendar_date','ptp')
#                )



xmas_ptps  = (xmas_ptps
                  .select(substring(xmas_ptps.Product_Sub_Group_Code,0,4).alias('group'),'*')
                  .withColumnRenamed("Product_Sub_Group_Code","product")
                  .withColumn('ptp',xmas_ptps['ptp'].cast(DoubleType()))
                  .orderBy(asc("Calendar_Date"))
                 )

print xmas_ptps.count()
print xmas_ptps.printSchema
xmas_ptps_pandas = xmas_ptps.toPandas()

#xmas_ptps_pandas.hierarchy_group.str.strip()
#bonfireNightPTPs_pandas
#nSubgrp = bonfireNightPTPs_pandas
print xmas_ptps_pandas.shape
xmas_ptps_pandas

In [ ]:
xmas_ptps_pandas['group'].unique()

In [ ]:

years_vec = decomp.keys()
method = 'median'

#% years_vec = [2012 2013 2014 ];
plot_figs = False #% 0 to disable plotting
#% a temporary and random file load, to extract some parameters

group_names = decomp[years_vec[0]].keys()

events  = decomp[years_vec[0]][group_names[0]]['events']

# collect all subgroups names
subgroups_names = list()
for  g in group_names:
            subgroups_names =subgroups_names + list(decomp[years_vec[0]][g]['products'])

stores =  decomp[years_vec[0]][group_names[0]]['stores']
nStores =len(stores)
dateVec =  decomp[years_vec[0]][group_names[0]]['dateVec']
print len(subgroups_names)
print len(dateVec)

In [ ]:

nSubgrp = len(subgroups_names) 
nDays = decomp[years_vec[0]][decomp[years_vec[0]].keys()[0]]['E_BV'].shape[1]

# initializing store_day_subgr_yr_tensor
store_day_subgr_yr_event_tensor = np.zeros((nStores,nDays,nSubgrp,len(years_vec)))
store_day_subgr_yr_basis_tensor = np.zeros((nStores,nDays,nSubgrp,len(years_vec)))
store_day_subgr_yr_ptp_tensor   = np.zeros((nStores,nDays,nSubgrp,len(years_vec)))
storeRange = np.zeros((nStores, nSubgrp, len(years_vec)))


# looping over all subgroups
subgroup_ind = 0
for subgroup in subgroups_names:
    yr_ind = 0
    print subgroup
    for y  in years_vec:
        print y
        #pdb.set_trace()
        
        #% Constructing the filename
        #filename = ['bonfireSubgroupDecomps', subgroups_names{subgroup_ind}, num2str(years_vec(yr_ind)), '.mat
        group_name = subgroup[:-1]
        dat = decomp[y][group_name]
        
        jj = np.where(decomp[y][group_name]['products'] == subgroup)[0]
        # Sales matrix (stores, days)
        
        X = dat['X'][:,:,jj]
        X = X.reshape((X.shape[0],X.shape[1]))
        #results = dat['results']
        T = dat['E_T'][:,:,jj]
        T  = T.reshape((T.shape[0],T.shape[1]))
        
        V = dat['E_V'][:,:,jj]
        V =  V.reshape((V.shape[0],V.shape[1]))
        
        N = dat['N'][:,:,jj]
        N =  N.reshape((N.shape[0],N.shape[1]))
        
        ptp = dat['ptp'][:,:,jj]
        ptp =  ptp.reshape((ptp.shape[0],ptp.shape[1]))
        
        
        #%% forming the base shapes
        n_basis = dat['basis_m'].shape[0]
        
        basis_inds_vec = range( 0, n_basis)
        T_basis = T[:,basis_inds_vec]
        V_basis = V[basis_inds_vec,:]
        basis_time = N * np.dot(T_basis , V_basis)
        
        # forming the promos
        n_promos = dat['promos'].shape[0]
        promos_inds_vec = range(n_basis, n_basis+n_promos)
        T_promos = T[:,promos_inds_vec]
        V_promos = V[promos_inds_vec,:]
        promos_time = N * np.dot(T_promos , V_promos)
        
        # forming the events
        events_inds_vec = range(n_basis+n_promos, V.shape[0] )
        T_events = T[:,events_inds_vec]
        V_events = V[events_inds_vec,:]
        events_time = N * np.dot(T_events , V_events)
        
        #%% including ptp
        X_hat = ptp * (basis_time + promos_time + events_time)
        
        #%% setting X-axis as date
                           
        #from datetime import datetime

        #date_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
        #date_1 = datetime.datetime.strptime(start_date, "%m/%d/%y")
        #end_date = date_1 + datetime.timedelta(days=10)
        #all_days = datetime.strptime(dat['startDate']) + datetime.timedelta(days=dat['nDays'])
        #all_days =                    
        #dat['endDate'] = all_days[-1] # % had disappeared, recalc'd
        #xData = linspace(dat.startDate,dat.endDate,size(X,2));
        
        #import datetime
        #all_days = range(datetime.datetime.strptime('2015-03-03',"%Y-%m-%d"),datetime.datetime.strptime('2015-03-03',"%Y-%m-%d") + datetime.timedelta(days=100))
        
        #all_days = [datetime.datetime.strptime('2015-03-03',"%Y-%m-%d") + datetime.timedelta(days=x) for x in range(0, dat['nDays'])]

        #xData = linspace(dat.startDate,dat.endDate,size(X,2)) # not sure what should it be replaced with
    
    
        store_day_subgr_yr_event_tensor[:,:,subgroup_ind,yr_ind] = events_time
        store_day_subgr_yr_basis_tensor[:,:,subgroup_ind,yr_ind] = basis_time
        store_day_subgr_yr_basis_tensor[:,:,subgroup_ind,yr_ind] = promos_time
        store_day_subgr_yr_ptp_tensor[:,:,subgroup_ind,yr_ind] = ptp
        
        if subgroup_ind == 0:
            storeRange_bases = np.zeros(( nStores, len(subgroups_names ), len(years_vec)))
            storeRange_events  = np.zeros(( nStores, len(subgroups_names ), len(years_vec)))
      
        storeRange_bases[:,subgroup_ind,yr_ind]  = np.all(store_day_subgr_yr_basis_tensor[:,:,subgroup_ind,yr_ind]==0, 1)
        storeRange_events[:,subgroup_ind,yr_ind] = np.all(store_day_subgr_yr_event_tensor[:,:,subgroup_ind,yr_ind]==0, 1)

        #% minuture version
     
        basis_store_days = store_day_subgr_yr_basis_tensor[:,:,subgroup_ind,yr_ind]
        event_store_days = store_day_subgr_yr_event_tensor[:,:,subgroup_ind,yr_ind]


        #% calculating nan years for all subgroups
        if subgroup_ind == 0:
            nanSubgroupYear_bases  = np.zeros(( len(subgroups_names ), len(years_vec)))
            nanSubgroupYear_events =  np.zeros(( len(subgroups_names ), len(years_vec)))
            
        nanSubgroupYear_bases[subgroup_ind,yr_ind] = np.sum(np.isnan(basis_store_days))  == basis_store_days.shape[0] * basis_store_days.shape[1]
        nanSubgroupYear_events[subgroup_ind,yr_ind] = np.sum(np.isnan(event_store_days)) == event_store_days.shape[0] * event_store_days.shape[1]


        #if y == 2015:
        #    if subgroup_ind == 0:
        #        salesLastYear  = np.zeros((X_hat.shape[0],X_hat.shape[1], len(subgroups_names )))
        #        salesLastYear[:,:,subgroup_ind] = X_hat
        #    else:
        #        salesLastYear[:,:,subgroup_ind] = X_hat
        
        yr_ind = yr_ind + 1
                
    subgroup_ind = subgroup_ind + 1
      
#nYearsMissing_bases = np.sum(storeRange_bases,3)
#[i_b,j_b] = np.where(nYearsMissing_bases == len(years_vec))
                                  
#list_of_allZeroYears_store_subgrp_fromBases = [i_b, j_b]

#nYearsMissing_events = np.sum(storeRange_events,3);
#[i_e,j_e] = np.where(nYearsMissing_events == len(years_vec))
                                  
#list_of_allZeroYears_store_subgrp_fromEvents = [i_e j_e]

In [ ]:
#%% initialize
#% nan_sg_yr = cell(1,nSubgrp);
event_subgroups = {}
basis_subgroups = {}
#% ptp_subgroups   = {cell(1,nSubgrp);}

#% seperate subgroups
subgroup_ind = 0
for subgroup in subgroups_names:
        #%nan_sg_yr{subgroup_ind} = squeeze(any(any(isnan(store_day_subgr_yr_basis_tensor(:,:,subgroup_ind,:)))));
        event_subgroups[subgroup] = np.squeeze(store_day_subgr_yr_event_tensor[:,:,subgroup_ind,:])
        basis_subgroups[subgroup] = np.squeeze(store_day_subgr_yr_basis_tensor[:,:,subgroup_ind,:])
        #%ptp_subgroups{subgroup_ind} = squeeze(store_day_subgr_yr_basis_tensor(:,:,subgroup_ind,:));
        subgroup_ind = subgroup_ind + 1


#%% prediction
#% initilize
event_pred_subgr_mat = {}  # % contains the predicted (in this case 'mean') event values
basis_pred_subgr_mat = {} # % contains the predicted (in this case 'mean') basis values
#%basis_pred_subgr_mat_no_ptp = cell(1, nSubgrp); % contains the predicted (in this case 'mean') basis values


#% calculate the predictions


for subgroup in subgroups_names:       
        if len(event_subgroups[subgroup].shape) == 2:
            event_pred_subgr_mat[subgroup]        =  event_subgroups[subgroup] 
            basis_pred_subgr_mat[subgroup]        =  basis_subgroups[subgroup] 
           
        else:  
        
            if method == 'mean':
                event_pred_subgr_mat[subgroup]        =  np.mean(event_subgroups[subgroup],2) 
                basis_pred_subgr_mat[subgroup]        =  np.mean(basis_subgroups[subgroup],2) 
            elif method == 'median':
                #pdb.set_trace()
                event_pred_subgr_mat[subgroup]        =  np.median(event_subgroups[subgroup],2) 
                basis_pred_subgr_mat[subgroup]        =  np.median(basis_subgroups[subgroup],2) 
            elif method == 'exp':
                alpha =  0.5
                w = exp_smoothing_average_weights(alpha,event_subgroups[subgroup].shape[2])
        
                event_pred_subgr_mat[subgroup]        =  np.average(event_subgroups[subgroup],2,weights= w) 
                basis_pred_subgr_mat[subgroup]        =  np.average(basis_subgroups[subgroup],2, weights= w) 
            else:
                print 'incorrect method is set'
        
        

#%     event_pred_subgr_mat{subgr_}        = mean(ptp_subgroups{subgroup_ind} .* event_subgroups{subgr_},3);
#%     basis_pred_subgr_mat{subgr_}        = mean(ptp_subgroups{subgroup_ind} .* basis_subgroups{subgr_},3);
#    %basis_pred_subgr_mat_no_ptp{subgr_} = mean(basis_subgroups{subgr_},3);

#% Exporting predicted event and basis function shapes for the APP

#% Replace NaNs with 0
for subgroup in subgroups_names:
        event_pred_subgr_mat[subgroup][np.isnan(event_pred_subgr_mat[subgroup])] = 0
        basis_pred_subgr_mat[subgroup][np.isnan(basis_pred_subgr_mat[subgroup])] = 0
        




In [ ]:
#exportDecomps draft
# generate CSV files
trialStores = xmas_ptps_pandas['Retail_Outlet_Number'].unique() # hardcode

unionTable = False

   
if forecast:
    forecast_date_start = '2016-11-28'
    forecast_date_end = '2017-01-15'
else:
    backtest_year
    forecast_date_start = str(backtest_year)+'-11-28'
    forecast_date_end = str(backtest_year+1)+'-01-15'

delta =  ((datetime.datetime.strptime(forecast_date_end ,"%Y-%m-%d")) -  
                    (datetime.datetime.strptime(forecast_date_start ,"%Y-%m-%d"))).days



date_range_index  =(( np.array(dateVec) >= datetime.datetime.strptime(str(dateVec[0].year)+'-11-28', "%Y-%m-%d") ) 
                            * (np.array(dateVec) <= datetime.datetime.strptime(str(dateVec[0].year+1)+'-01-15', "%Y-%m-%d")))
    
print np.sum(date_range_index)
#ptp_filename =  'D:\projects\forecasting\bonfireNightPTPs.csv';
#ptp_table = import_bonfireNightPTPs(ptp_filename);
#ptp_table.Product_Sub_Group_Code = strtrim(ptp_table.Product_Sub_Group_Code);

PTP_pandas = xmas_ptps_pandas

#print bonfireNightPTPs
#bonfireNightPTPs_df.show()

#bonfireNightPTPs_pandas.hierarchy_group.str.strip()

counter_hg =  0

dateVec_F = [(datetime.datetime.strptime(forecast_date_start ,"%Y-%m-%d")
                        + datetime.timedelta(days=x)).strftime('%Y-%m-%d') for x in range( delta +1)
          ]
       

#dateVecLY = [(datetime.datetime.strptime(forecast_date_start ,"%Y-%m-%d")
#                        + datetime.timedelta(days=x)).strftime('%Y-%m-%d') for x in range( delta  +1)
#            ]
      
for hg in xmas_ptps_pandas['product'].unique():
    
    print 'Loading subgroup '+hg 

    for st in trialStores:
        
        #pdb.set_trace()
        
        ptp2016 = PTP_pandas[PTP_pandas['Retail_Outlet_Number'] == st][PTP_pandas['product'] == hg]['ptp']
    
        baseSalesFcast     = ptp2016.T * basis_pred_subgr_mat[hg][ stores == st, date_range_index ]
        eventForecast      = ptp2016.T * event_pred_subgr_mat[hg][ stores == st, date_range_index ]
        
        baseSalesFcast[np.isnan(baseSalesFcast)] = 0
        eventForecast[np.isnan(eventForecast)] = 0
        
        
        
        xmasForecast          = np.zeros((eventForecast.shape))
        #halloweenForecast        = np.zeros((eventForecast.shape))
        xmasForecast[events[2,date_range_index] == 1  ] = eventForecast[events[2,date_range_index] == 1 ]
        #halloweenForecast[0: 5 ]  = eventForecast[0: 5 ]
        
        #ptp2015 = np.vstack( ptp2016[5:-1],  ptp2016[0:5])
        
        # not sure what is meant here -  will need to coordinate with Bobby to proceed
        #salesLY = ptp2015.T * salesLastYear[ stores == st, forecast_date_range, counter_hg]
        #salesLY[np.isnan(salesLY)] = 0
        
        baseTable = pd.DataFrame(np.hstack( (
                                          np.tile(st, (delta + 1,1)),
                                          np.tile(hg,(delta + 1,1)),
                                          np.zeros((delta + 1,1)),
                                          np.array(dateVec_F).reshape(len(dateVec_F),1),
                                          baseSalesFcast.T.reshape((baseSalesFcast.T.shape[0],1)),
                                          np.tile('?',(delta + 1, 1)),
                                          np.zeros((delta + 1, 1)),
                                          np.tile('?',(delta + 1, 1)),
                                          np.tile('?',(delta + 1, 1))) ),
                                          columns = [ 'Store','hierarchy_code','zero1','calendar_date',
                                                    'forecast','qMark1','zero2','qMark2','qMark3'])
        
        
        
        xmasTable = pd.DataFrame(np.hstack( (
                                        np.tile(5,( delta + 1, 1)),
                                        np.tile(st,( delta + 1, 1)),
                                        np.tile(hg, (delta + 1, 1)),
                                        np.zeros((delta + 1, 1)),
                                        np.array(dateVec_F).reshape(len(dateVec_F),1),
                                        xmasForecast.T.reshape((xmasForecast.T.shape[0],1)),
                                        np.tile('?',(delta + 1, 1)),
                                        np.zeros((delta + 1,1)),
                                        np.tile('?',(delta + 1, 1)),
                                        np.tile('?',(delta + 1, 1)) ) ),
                                        columns =  [ 'Event_id', 'store','hierarchy_code',
                                                    'zero1','calendar_date','forecast','qMark1','zero2','qMark2','qMark3'])
        
#        halloweenTable = pd.DataFrame(np.hstack(
#                                        np.tile(4, (delta + 1, 1)),
#                                        np.tile(st, (delta + 1, 1)),
#                                        np.tile(hg, (delta + 1, 1)),
#                                        np.zeros((delta + 1, 1)),
#                                        dateVec,
#                                        halloweenForecast.T,
#                                        np.tile('?',(delta + 1, 1)),
#                                        np.zeros((delta + 1,1)),
#                                        np.tile('?',(delta + 1, 1)),
#                                        np.tile('?',(delta + 1, 1)),
#                                        columns =  [ 'Event_id', 'store','hierarchy_code',
#                                                    'zero1','date','forecast','qMark1','zero2','qMark2','qMark3']))
        
        #eventsTable = union(bonfireTable, halloweenTable);
        #eventsTable = bonfireTable.append(halloweenTable, ignore_index=True)
        
        #LYTable = pd.DataFrame(np.hstack(
        #                            np.tile(st, (delta + 1,1)),
        #                            np.tile(hg, (delta + 1,1)),
        #                            np.zeros((delta + 1,1)),
        #                            dateVecLY,
        #                            salesLY.T,
        #                            columns = [ 'Store','hierarchy_code','zero','date','sales']))
        
        
        if  unionTable:
            baseTableOut = baseTable.append(baseTableOut, ignore_index=True)
            xmasTableOut = xmasTable.append(xmasTableOut, ignore_index=True)
        #    LYTableOut = LYTable.append(LYTableOut, ignore_index=True )
        else:
            baseTableOut = baseTable
            xmasTableOut = xmasTable
        #    LYTableOut = LYTable
            
            unionTable = True

        #             dcmp.utils.bnmf_diagnostics(results, X./results.ptp, X, zeros(size(X)), zeros(nStores,np), ones(np, nDays), 1:nBasis+1);
 
    counter_hg = counter_hg + 1


In [10]:
stop here

SyntaxError: invalid syntax (<ipython-input-10-f9ef205a8f26>, line 1)

In [11]:

#tempOutputTable = 'zg43_test_bonfire_data';

xmas_baseTable_filename = ('/insight_labs/retaildemandforecasting/data/xmas/output/'
                           +str(num_of_stores)+
                           '_'+the34_23
                           +'_xmasBaseTableCSV_2111_2015'
                           +('forecast' if forecast else 'backtest')
                            +'_'
                            +method
                          )

xmas_eventTable_filename = ('/insight_labs/retaildemandforecasting/data/xmas/output/'
                            +str(num_of_stores)
                            +'_'+the34_23
                            +'_xmasEventsTableCSV_2111_2015'
                            +('forecast' if forecast else 'backtest')
                            +'_'
                            +method)
#bonfire_LYTable_filename = 'bonfireLYTableCSV.txt';
#bonfire_baseTable_pipe_filename = 'bonfireBaseTableCSV_pipe.txt';
#bonfire_eventTable_pipe_filename = 'bonfireEventsTableCSV_pipe.txt';
#bonfire_LYTable_pipe_filename = 'bonfireLYTableCSV_pipe.txt';
#formatOut = 'dd_mm_yyyy__HH_MM';
#time_str=datestr(now,formatOut);
#folder_name = ['bonfire_',time_str];
#mkdir(folder_name)
#cd(folder_name)

#% writetable(baseTableOut,fullfile(dataRoot,bonfire_baseTable_filename));
#% writetable(eventsTableOut,fullfile(dataRoot,bonfire_eventTable_filename));
#% writetable(LYTableOut,fullfile(dataRoot,bonfire_LYTable_filename));

(sqlContext
        .createDataFrame(baseTableOut)
        .coalesce(1)
        .write
        .format("com.databricks.spark.csv")
        .option("header","false")
        .save(xmas_baseTable_filename) )
        
(sqlContext
        .createDataFrame(xmasTableOut)
        .coalesce(1)
        .write
        .format("com.databricks.spark.csv")
        .option("header","false")
        .save(xmas_eventTable_filename) )
        
        
#(sqlContext
#        .createDataFrame(LYTableOut)
#        .coalesce(1)
#        .write
#        .format("com.databricks.spark.csv")
#        .option("header","true")
#        .save(bonfire_LYTable_filename))



In [ ]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import StringType


xmasTableOut_df = (sqlContext
                     .createDataFrame(xmasTableOut)
                   )

xmasTableOut_df = xmasTableOut_df.withColumn("Event_id", xmasTableOut_df["Event_id"].cast(IntegerType()))
xmasTableOut_df = xmasTableOut_df.withColumn("store", xmasTableOut_df["store"].cast(IntegerType()))
xmasTableOut_df = xmasTableOut_df.withColumn("zero1", xmasTableOut_df["zero1"].cast(DoubleType()))
xmasTableOut_df = xmasTableOut_df.withColumn("calendar_date", xmasTableOut_df["calendar_date"].cast(DateType()))
xmasTableOut_df = xmasTableOut_df.withColumn("forecast", xmasTableOut_df["forecast"].cast(DoubleType()))
xmasTableOut_df = xmasTableOut_df.withColumn("zero2", xmasTableOut_df["zero2"].cast(DoubleType()))
xmasTableOut_df = xmasTableOut_df.withColumn("hierarchy_code", xmasTableOut_df["hierarchy_code"].cast(StringType("varchar")))

xmasTableOut_df = xmasTableOut_df.withColumn("rowId_for_primary_key", monotonically_increasing_id()) 


                                                                  
xmasTableOut_df.cache()

xmasTableOut_df.printSchema()
xmasTableOut_df.show(5)


In [ ]:
#lets test writign into TRD table 

prop = {
    "user": "GCT8",
    "password": "ludo4ka",
    "driver" : "com.teradata.jdbc.TeraDriver"
}

(xmasTableOut_df.select(xmasTableOut_df.rowId_for_primary_key,
                        xmasTableOut_df.Event_id,
                        xmasTableOut_df.store,
                        xmasTableOut_df.zero1,
                        xmasTableOut_df.calendar_date,
                        xmasTableOut_df.forecast,
                        xmasTableOut_df.qMark1,
                        xmasTableOut_df.zero2
                        
                        
                       )
                 .write
                 .jdbc(url  = "jdbc:teradata://TDPS.ukroi.tesco.org/DATABASE=DXWI_PROD_CRORDER_PLAY_PEN",
                       table = "xmas_Events_2111",
                       mode = "overwrite",
                       properties  = prop
                      )
)
      
 

In [ ]:
# sqopp way
!sqoop export --connect jdbc:teradata://TDPS.ukroi.tesco.org/DATABASE=DXWI_PROD_CRORDER_PLAY_PEN --username GCT8 --password ludo4ka --table xmas_BackTestEvents_2111_1 --export-dir /insight_labs/retaildemandforecasting/data/xmas/output/800_2_3_xmasEventsTableCSV_2111_2015forecast_median  --input-fields-terminated-by ',' --lines-terminated-by '\n' --m 1

In [ ]:
#lets test writign into TRD table 

(sqlContext
                .createDataFrame(xmasTableOut)
                .coalesce(1)
                .write
                .format('jdbc')
                .options(url="jdbc:teradata://TDPS.ukroi.tesco.org/DATABASE=DXWI_PROD_CRORDER_PLAY_PEN",
                     dbtable="xmas_BackTestEvents_2111",
                     user = "GCT8",
                     password = "ludo4ka",
                     driver = "com.teradata.jdbc.TeraDriver")
                .save()
             )

In [ ]:
stop_here

In [ ]:
sc.stop()

In [ ]:
df = (sqlContext.read.format('jdbc')
        .options(url="jdbc:teradata://TDPS.ukroi.tesco.org/DATABASE=DXWI_PROD_CRORDER_PLAY_PEN",
                 dbtable="zg43_xmasPTPs",
                 user = "GCT8",
                 password = "ludo4ka",
                 driver = "com.teradata.jdbc.TeraDriver")
        .load()
     )



#jdbcDF = sqlContext.read.format("jdbc").options("url" = "TDPS.ukroi.tesco.org/DATABASE=DXWI_PROD_CRORDER_PLAY_PEN",
#   "user" -> "GCT8",
#   "password" -> "ludo4ka",
#   "dbtable" -> s"($query) as tbl",
#   "driver" -> "com.teradata.jdbc.TeraDriver"
# )).load()

In [ ]:
def exp_smoothing_average_weights(alpha,dim):
    a = np.zeros(dim)
    for i in range(dim):
        a[i] = alpha * ((1-alpha) ** i)
    return a

In [ ]:
# exponential smoothing
def exp_smoothing_average(alpha,x,axis):
    if axis.size == 0:
        if len(x) >1:
            s_t  = alpha*x[-1] + (1 - alpha)*exp_smoothing_average(alpha,x[0:-1])
        else:
            s_t = x[0]
    elif axis = 0:
        s_t = np.zeros(x.shape[0])
        for i in range(x.shape[0]):
            s_t[i] = exp_smoothing_average(alpha,x[i,:],[])
 
            
    return s_t
